In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [3]:
#load pages of the docx file
loader_docx = Docx2txtLoader("Introduction_to_Data_and_Data_Science_2.docx")
pages = loader_docx.load()


#splitted based on the course and lecture titles
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on = [('#', 'Course Title'),
                                                                ('##', 'Lecture Title')] )


#removed the new line characters from all the text 
pages_md_split = md_splitter.split_text(pages[0].page_content)

for i in range(len(pages_md_split)):
    pages_md_split[i].page_content = ' '.join(pages_md_split[i].page_content.split())


#splitted based on the number of characters using the chunk, overlap and a period as a separator
char_splitter = CharacterTextSplitter(
    separator = '.',
    chunk_size = 500,
    chunk_overlap = 50
)

pages_char_split = char_splitter.split_documents(pages_md_split)


#created an instance of the openais embbedding class with the desired embedding model
embedding = OpenAIEmbeddings(model = 'text-embedding-ada-002')

In [4]:
len(pages_char_split) # nr of docs sorted in pages  

20

In [5]:
vectorstore = Chroma.from_documents(
    documents=pages_char_split, 
    embedding=embedding, 
    persist_directory="./intro-to-ds-lectures")


In [6]:
vectorstore_from_directory = Chroma(persist_directory = "./intro-to-ds-lectures", embedding_function = embedding)

/var/folders/jw/c29_qvvs3yb90jqlkdbpt4_m0000gn/T/ipykernel_5950/3089742234.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_from_directory = Chroma(persist_directory = "./intro-to-ds-lectures", embedding_function = embedding)


In [7]:
pages_char_split[19]

Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}, page_content='Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching!')